In [ ]:
import nest
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
nest.ResetKernel()
nest.resolution = 0.1

# Create Gamma input
g = nest.Create("sinusoidal_gamma_generator",
    params={
        "rate": 100.0,
        "amplitude": 90.0,
        "frequency": 50.0,
        "phase": 0.0,
        "order": 3.0,
        "individual_spike_trains": True,
    })   # note the syntax for different order parameter of the two nodes

# Create noise generators (E and I)
noise = nest.Create('poisson_generator', 2,
                    [{'rate': 70000.0}, {'rate': 20000.0}])
voltmeter = nest.Create('voltmeter', params={'interval': 1.})


pyr1spk = nest.Create('spike_recorder', params={'label': 'pyr1'})
pyr2spk = nest.Create('spike_recorder', params={'label': 'pyr2'})
pv1spk = nest.Create('spike_recorder', params={'label': 'pv1'})
pv2spk = nest.Create('spike_recorder', params={'label': 'pv2'})
s = nest.Create('spike_recorder', params={'label': 's'})

pops = []                                  # Container for Pops

# E Pop:
pyr_1 = nest.Create('iaf_psc_alpha', 500)
nest.Connect(voltmeter, pyr_1)
pops.append(pyr_1)
nest.Connect(pyr_1, pyr1spk)
pyr_2 = nest.Create('iaf_psc_alpha', 500)
nest.Connect(pyr_2, pyr2spk)
pops.append(pyr_2)

# I Pop
pv_1 = nest.Create('iaf_psc_alpha', 250)
nest.Connect(pv_1, pv1spk)
pops.append(pv_1)
pv_2 = nest.Create('iaf_psc_alpha', 250)
nest.Connect(pv_2, pv2spk)
pops.append(pv_1)

# Connect Gamma to input layer
nest.Connect(g, pyr_1)
nest.Connect(g, pv_1)
nest.Connect(g, s)

# Connect noise to all
for pop in pops:
    nest.Connect(noise[:1], pop, syn_spec={'weight': 2.0, 'delay': 1.0})
    nest.Connect(noise[1:], pop, syn_spec={'weight': -5.0, 'delay': 1.0})


# Connect Pops
nest.Connect(pyr_1, pyr_2, syn_spec={'weight': nest.random.lognormal(mean=1.0, std=0.2), 'delay': 1.0})
nest.Connect(pyr_1, pv_2, syn_spec={'weight': nest.random.lognormal(mean=1.0, std=0.2), 'delay': 1.0})
nest.Connect(pv_1, pyr_1, syn_spec={'weight': nest.random.lognormal(mean=-5.0, std=1.0), 'delay': 0.75})
nest.Connect(pv_1, pyr_2, syn_spec={'weight': nest.random.lognormal(mean=-5.0, std=1.0), 'delay': 0.75})

nest.Simulate(1000)

nest.voltage_trace.from_device(voltmeter)